In [2]:
import psycopg2
import pandas as pd
from dateutil import parser

In [3]:
# Database connection details
db_config = {
    'dbname': 'cis_566_database',
    'user': 'postgres',
    'password': '',
    'host': 'localhost',
    'port': '5432'
}

In [ ]:
file_path = 'Datasets/car_prices.csv'  # Replace with the actual path to your cleaned CSV file
data = pd.read_csv(file_path)

In [ ]:
def convertDate(date_str):
    # Check if date_str is NaN or empty
    if pd.isna(date_str) or not isinstance(date_str, str) or date_str.strip() == "":
        return None
    
    try:
        parsed_date = parser.parse(date_str)
        db_date = parsed_date.isoformat()
        print(db_date)
        return db_date
    except (ValueError, TypeError):
        return None

In [ ]:
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

In [ ]:
def insertDataToDB():
    try:
        print(conn, cursor)
        for _, row in data.iterrows():
            print(_, row)
            cursor.execute(
                """
                INSERT INTO Vehicles (vin, year, make, model, trim, body, transmission, color, interior)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                ON CONFLICT (vin) DO NOTHING;
                """,
                (row['vin'], row['year'], row['make'], row['model'], row['trim'], row['body'], row['transmission'], row['color'], row['interior'])
            )
        conn.commit()
          

        for _, row in data.iterrows():
            print(_, row)
            saleDate = convertDate(row['saledate'])
            cursor.execute(
                """
                INSERT INTO Sales (vin, state, saledate, sellingprice)
                VALUES (%s, %s, %s, %s)
                ON CONFLICT DO NOTHING;
                """,
                (row['vin'], row['state'], saleDate, row['sellingprice'])
            )
            conn.commit()

        for _, row in data.iterrows():
            print(_, row)
            cursor.execute(
                """
                INSERT INTO MarketTrends (vin, mmr)
                VALUES (%s, %s)
                ON CONFLICT DO NOTHING;
                """,
                (row['vin'], row['mmr'])
            )
        conn.commit()

        for _, row in data.iterrows():
            print(_, row)
            cursor.execute(
                """
                INSERT INTO Condition (vin, condition_rating, odometer)
                VALUES (%s, %s, %s)
                ON CONFLICT DO NOTHING;
                """,
                (row['vin'], row['condition'], row['odometer'])
            )
        conn.commit()

    except Exception as e:
        print(f"Error: {e}")

In [ ]:
# Insert data to db from csv file
insertDataToDB()

In [ ]:
# Delete all the relations of the vehicles for the discrepencies 
query_0 = """
  delete from sales where vin in (select vin from vehicles where make = 'NaN' or model = 'NaN');
  delete from markettrends where vin in (select vin from vehicles where make = 'NaN' or model = 'NaN');
  delete from "condition" where vin in (select vin from vehicles where make = 'NaN' or model = 'NaN');
  delete from vehicles where make = 'NaN' or model = 'NaN';
"""
conn.execute(query_0)
conn.commit()

In [ ]:
# Clear null and faulty data from sales table using salesdate
query_1 = "DELETE FROM sales WHERE CAST(saledate AS TEXT) NOT ILIKE '2%' and CAST(saledate AS TEXT) NOT ILIKE '1%';"
conn.execute(query_1)
conn.commit()

In [ ]:
# Filtered markettrends data to remove unknown mmrs
query_2 = "delete from markettrends where mmr = 'NaN'"
conn.execute(query_2)
conn.commit()

In [ ]:
# Set the odometer ratings to NULL for the faulty data
query_3 = "UPDATE condition SET odometer = NULL where odometer = 'NaN'"
conn.execute(query_3)
conn.commit()

In [ ]:
# Clear null and faulty data from sales table
query_4 = "delete from sales where sellingprice = 'NaN' or is null"
conn.execute(query_4)
conn.commit()